In [1]:
# AWS EC2 UDP Testing

# UDP CODE EXAMPLE

import time
from socket import *
serverName = 'ec2-35-90-59-237.us-west-2.compute.amazonaws.com' # AWS Server Public DNS
serverPort = 1113 # TEST UDP Port

In [3]:

clientSocket = socket(AF_INET, SOCK_DGRAM)
message = '1'

for i in range(5):
    print('sending message: {}'.format(i))
    clientSocket.sendto(message.encode(),(serverName, serverPort))
    time.sleep(1)
clientSocket.close()

sending message: 0
sending message: 1
sending message: 2
sending message: 3
sending message: 4


In [3]:
# AWS EC2 TCP Process Testing
# Testing for Connection Status Reporting via TCP Client Connection:

import time
import socket
import subprocess
import multiprocessing
import pynq.lib.rgbled as rgbled
from pynq.overlays.base import BaseOverlay

TCPserver = 'ec2-35-90-59-237.us-west-2.compute.amazonaws.com' # AWS Server Public DNS
clientPort = 11114
port = 11115

totalMin = 2
timerVal = totalMin * 60

manager = multiprocessing.Manager()
seq_complete_list = manager.list()

def client_process():
    # Create a TCP/IP socket
    c_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    print('Connecting Client. . .')
    rgbled.RGBLED(1).write(0x0) # Turn off RGB LED
    rgbled.RGBLED(1).write(0x2) # RGB LED Green
    # Connect Client
    c_sock.connect((TCPserver, clientPort))
    print('Connected to Server!')
    upMessage = 'up'
    downMessage = 'down'
    
    while True:
        time.sleep(2)
        c_sock.send(upMessage.encode())
        if len(seq_complete_list) > 0:
            rgbled.RGBLED(1).write(0x0) # Turn off RGB LED
            rgbled.RGBLED(1).write(0x5) # RGB LED Magenta
            c_sock.send(downMessage.encode())
            break
    # Disconnect from Client
    c_sock.close()
    print('Client Connection Socket Closed!')
    seq_complete_list.append(1)

    

def main():  
    # Create 1 process
    p1 = multiprocessing.Process(target=client_process)
    
    # Start both processes
    p1.start()
    
    for t in range(50):
        time.sleep(2)
    seq_complete_list.append(1)
    
    # Give sufficient time for socket to close 
    time.sleep(5)
    
    # Join process
    p1.join()
    p1.close()
    print('Sequence Stopped!')
    rgbled.RGBLED(1).write(0x0) # Turn off RGB LED


if __name__ == '__main__':
    main()


Connecting Client. . .
Connected to Server!
Client Connection Socket Closed!
Sequence Stopped!
